In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

playground_series_s5e6_path = kagglehub.competition_download('playground-series-s5e6')

print('Data source import complete.')

100%|██████████| 11.7M/11.7M [00:00<00:00, 161MB/s]

Extracting files...


Data source import complete.


In [3]:
#import the necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder #for model selection strategies (like cross validation and data preprocessing)
import lightgbm as lgb # for building high-performance gradient boosting models, optimized for large datasets, low memory usage, and faster training

from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier

In [4]:
train = pd.read_csv(f'{playground_series_s5e6_path}/train.csv')
test = pd.read_csv(f'{playground_series_s5e6_path}/test.csv')

print(train.shape)
print(test.shape)

(750000, 10)
(250000, 9)


In [6]:
# Separate features (X) and target (y)
X = train.drop('Fertilizer Name', axis=1)
y = train['Fertilizer Name']

# Although not used in this specific cell, it's good practice to also prepare the test set features
X_test = test.drop('id', axis=1) # Assuming 'id' is not a feature for the model

# For now, we'll use the entire training set as X_train and y_train
# If you need to split into training and validation sets, you can use train_test_split from sklearn.model_selection
X_train = X
y_train = y


print(X_train.dtypes)
print(y_train.dtype)

id              int64
Temparature     int64
Humidity        int64
Moisture        int64
Soil Type      object
Crop Type      object
Nitrogen        int64
Potassium       int64
Phosphorous     int64
dtype: object
object


In [8]:
#test cols
print(train.columns.tolist())

['id', 'Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type', 'Nitrogen', 'Potassium', 'Phosphorous', 'Fertilizer Name']


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['target_enc'] = le.fit_transform(train['Fertilizer Name'])


In [ ]:
print(le.classes_)


In [ ]:
features = [c for c in train.columns if c not in ['id', 'Fertilizer Name', 'target_enc']]
X = train[features]
y = train['target_enc']


In [ ]:
# 4. Set up cross-validation & LightGBM
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_probs = np.zeros((len(train), le.classes_.size))
test_probs = np.zeros((len(test), le.classes_.size))

# One-hot encode categorical features
X = pd.get_dummies(train[features], columns=['Soil Type', 'Crop Type'])
test_encoded = pd.get_dummies(test[features], columns=['Soil Type', 'Crop Type'])

# Align columns between training and test sets after one-hot encoding
train_cols = X.columns
test_cols = test_encoded.columns

missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    test_encoded[c] = 0

missing_in_train = set(test_cols) - set(train_cols)
for c in missing_in_train:
    X[c] = 0

test_encoded = test_encoded[train_cols]


for fold, (tr_idx, val_idx) in enumerate(kf.split(X)):
    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    train_data = lgb.Dataset(X_tr, label=y_tr)
    val_data = lgb.Dataset(X_val, label=y_val)

    model = lgb.train(
        params={'objective':'multiclass', 'num_class':le.classes_.size,
                'metric':'multi_logloss', 'learning_rate':0.1, 'verbosity':-1},
        train_set=train_data, valid_sets=[val_data],
        num_boost_round=200, callbacks=[lgb.early_stopping(stopping_rounds=20, verbose=50)]
    )

    oof_probs[val_idx] = model.predict(X_val)
    test_probs += model.predict(test_encoded) / kf.n_splits

Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[200]	valid_0's multi_logloss: 1.92258
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[200]	valid_0's multi_logloss: 1.9224
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[200]	valid_0's multi_logloss: 1.92287
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[200]	valid_0's multi_logloss: 1.92162


In [23]:
test_preds = test_probs
pd.DataFrame(test_preds).head()

,0,1,2,3,4,5,6
0,0.142648,0.149246,0.131921,0.136926,0.147689,0.159198,0.132372
1,0.148147,0.142488,0.191081,0.161420,0.138679,0.094995,0.123189
2,0.157401,0.158167,0.126242,0.169272,0.140842,0.138698,0.109379
3,0.140640,0.204576,0.154950,0.129843,0.111426,0.122054,0.136512
4,0.147389,0.123827,0.149747,0.199764,0.140946,0.108241,0.130085


In [32]:
def top3(preds):
    return le.inverse_transform(np.argsort(preds)[::-1][:3])

submission = pd.DataFrame([
    [i, " ".join(top3(row))] for i, row in zip(test_ids, test_preds)
], columns=["id", "Fertilizer Name"])
submission.to_csv("submission.csv", index=False)



In [33]:
print(top3(test_preds[0]))

['DAP' '14-35-14' '28-28']


In [34]:
submission.head()

,id,Fertilizer Name
0,750000,DAP 14-35-14 28-28
1,750001,17-17-17 20-20 10-26-26
2,750002,20-20 14-35-14 10-26-26
3,750003,14-35-14 17-17-17 10-26-26
4,750004,20-20 17-17-17 10-26-26
